In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Activation,Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import cross_val_score

dfTrain = pd.read_pickle("./trainingSet")
dfTrain = dfTrain.drop(columns=['DMA'])

dfTest = pd.read_pickle("./trainingSet")
dfTest = dfTest.drop(columns=['DMA'])

df.head()

,Flow,dev,t-4dev,t-8dev,t-12dev,t-16dev,t-20dev,t-24dev,t-28dev,t-32dev,t-36dev,t-40dev,t20Leak,t24Leak,t28Leak,t32Leak,t36Leak,t40Leak,LeakFixIn4Days
0,0.089,0.026370,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,False,False,False,False,False,False,False
1,0.033,-0.029000,0.029451,-0.012333,-0.018000,0.023451,0.047627,0.027000,0.018451,-0.010098,-0.005000,-0.001333,False,False,False,False,False,False,False
2,0.067,-0.006370,0.097098,-0.053370,0.048630,-0.009314,-0.015118,0.093630,-0.013902,0.034902,-0.024925,0.002630,False,False,False,False,False,False,False
3,0.111,0.039222,0.036569,-0.003815,-0.004778,0.029627,-0.023275,0.128222,0.025569,-0.009941,0.023528,0.018185,False,False,False,False,False,False,False
4,0.078,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,-0.010922,False,False,False,False,False,False,False


In [63]:
def df_to_numpy_array(df):
    ''' Seperate categorical and numerical columns of dataframe
        Input df: Pandas dataframe
        Output: numpy array
    '''
    #get names of numerical columns
    num_columns = list(df.select_dtypes(include=['float64', 'int64']).columns.values)
    
    #get numerical values into NumPy array
    num_values = df[num_columns].values
    
    #get categorical columns 
    bool_columns = list(df.select_dtypes(include=['bool']).columns.values)

    #transform categorical columns into numpy array
    bool_values = df[bool_columns].values
    
    return np.concatenate((bool_values,num_values),axis=1)

In [64]:
def model():
    model = Sequential()
    model.add(Dense(18, input_dim=18, init='normal', activation='relu'))
    model.add(Dense(18, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [65]:
XTrain = df_to_numpy_array(dfTrain.drop(["LeakFixIn4Days"],axis=1))
XTest = df_to_numpy_array(dfTest.drop(["LeakFixIn4Days"],axis=1))
yTrain = dfTrain['LeakFixIn4Days'].values
yTest = dfTest['LeakFixIn4Days'].values

In [66]:
XTrain[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.089     ,  0.02637037,  0.01460784, -0.02087037,
       -0.01862963,  0.01476471,  0.03127451,  0.01537037,  0.00360784,
       -0.01092157, -0.00550943, -0.03287037])

In [68]:
kr = KerasRegressor(build_fn=model, nb_epoch=1, batch_size=5, verbose=1)

scores = cross_val_score(kr, XTrain, yTrain, cv=4)
print("msle = %4.2f std = %4.2f" % (scores.mean(),scores.std()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(18, input_dim=18, activation="relu", kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(18, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  """


Epoch 1/1
184778/184778 [==============================] - 15s 81us/step
Epoch 1/1
184778/184778 [==============================] - 15s 82us/step
Epoch 1/1
184778/184778 [==============================] - 15s 82us/step
Epoch 1/1
184777/184777 [==============================] - 15s 83us/step
msle = -0.14 std = 0.01


In [71]:
    model = Sequential()
    model.add(Dense(18, input_dim=18, init='normal', activation='relu'))
    model.add(Dense(18, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(XTrain, yTrain, validation_data=(XTest,yTest), epochs=3, batch_size=5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(18, input_dim=18, activation="relu", kernel_initializer="normal")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(18, activation="relu", kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  after removing the cwd from sys.path.


Train on 739111 samples, validate on 739111 samples
Epoch 1/3
739111/739111 [==============================] - 165s 224us/step - loss: 0.1436 - acc: 0.9665 - val_loss: 0.1397 - val_acc: 0.9673
Epoch 2/3
739111/739111 [==============================] - 165s 223us/step - loss: 0.1393 - acc: 0.9672 - val_loss: 0.1394 - val_acc: 0.9673
Epoch 3/3
739111/739111 [==============================] - 165s 223us/step - loss: 0.1390 - acc: 0.9672 - val_loss: 0.1382 - val_acc: 0.9674
